In [ ]:
# import numpy as np
# import tkinter as tk
# from tkinter import filedialog
# import pandas as pd

# root = tk.Tk()
# root.withdraw()

# # file_path = filedialog.askopenfilename(
# #   title="Select geometry xyz file.",
# #   initialdir='.',
# #   filetypes=(
# #       # ("jpeg files", "*.jpg"),
# #       # ("csv files", "*.csv"),
# #       ("xyz files", "*.xyz"),
# #       ("gif files", "*.gif*")
# #       # ("png files", "*.png")
# #     )
# #   )

file_path = 'C:/Users/edu51/MJ_Python/rcm-python/cp6t6_0p_blyp35_geom.xyz'

from rdkit import Chem
a = Chem.rdmolfiles.MolFromXYZFile(file_path)


# # print(file_path)

In [19]:
import pandas as pd
import numpy as np

In [ ]:
import rcm

In [ ]:
rcm.B(0,1,2,3,4,0,1,2,4,5,2)

In [43]:
file_path_connectivity_table = 'cp6t6_0p_blyp35_geom_connectivity_global.csv'


df = pd.read_csv(
  file_path_connectivity_table, header=None, 
  names = ['current_weight', 'start','end']
)


display(df.head())

,current_weight,start,end
0,0.50,79,81
1,0.25,81,85
2,0.25,85,93
3,0.25,93,95
4,0.25,81,86


In [44]:
def count_iterable(i):
    return sum(1 for e in i)

In [45]:
count_iterable(a.GetAtoms())

480

In [46]:
df_xyz = pd.DataFrame(np.nan, columns=['atom', 'x', 'y', 'z'], index=np.arange(count_iterable(a.GetAtoms())))
# df

In [47]:
# b = Chem.MolToXYZBlock(a)
# df_depo = pd.DataFrame()

for i,atom in enumerate(a.GetAtoms()): 
  positions = a.GetConformer().GetAtomPosition(i)
  # print(atom.GetAtomicNum())
  df_xyz.at[i,'atom'] = atom.GetSymbol()
  df_xyz.at[i,'x'] = positions.x
  df_xyz.at[i,'y'] = positions.y
  df_xyz.at[i,'z'] = positions.z
  # print(atom.GetSymbol(), positions.x, positions.y, positions.z)
  
  # lst = [atom.GetSymbol(), positions.x, positions.y, positions.z]
  # df_depo = pd.concat([df_depo,[atom.GetSymbol(), positions.x, positions.y, positions.z]], axis=0)
  # df_depo.at[1,::] = lst
  # break

In [48]:
df

,current_weight,start,end
0,0.50,79,81
1,0.25,81,85
2,0.25,85,93
3,0.25,93,95
4,0.25,81,86
...,...,...,...
193,1.00,80,78
194,1.00,78,76
195,1.00,76,75
196,1.00,75,77


In [66]:
df_xyz.iloc[df.loc[0,'start'],::]

atom            C
x      -12.834856
y        1.080083
z        0.008206
Name: 79, dtype: object

In [57]:
df.loc[0,'current_weight']


0.5

In [54]:
df.loc[0,'start']

79

In [56]:
df.loc[0,'end']


81

In [51]:
for i in df.index:
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197


In [49]:
df_xyz

,atom,x,y,z
0,C,12.834420,-0.431648,1.263983
1,C,13.087348,-1.095274,2.516019
2,C,13.080758,-0.132733,3.471963
3,C,12.782510,1.115903,2.809385
4,N,12.655821,0.898976,1.462114
...,...,...,...,...
475,C,-2.373974,-7.610207,1.155282
476,H,-2.984208,-6.862316,-2.080746
477,H,-3.777700,-9.203738,-1.991108
478,H,-2.833165,-9.541487,1.988823


In [ ]:
positions

In [ ]:
pd.DataFrame([atom.GetSymbol(), positions.x, positions.y, positions.z])

In [ ]:
# import rcm

# a = rcm.Bz(0,1,2,3,0,7,8,6,8,9,15,5)
# print(a)
# a = rcm.Bx(0,1,2,3,0,7,8,6,8,9,15,5)
# print(a)
# a = rcm.By(0,1,2,3,0,7,8,6,8,9,15,5)
# print(a)
# a = rcm.B(0,1,2,3,0,7,8,6,8,9,15,5)
# print(a)

In [ ]:
file_path_list_of_spectator_atoms = 'cp6t6_0p_blyp35_geom_atoms.csv'

In [ ]:
import csv
with open(file_path_list_of_spectator_atoms, mode ='r')as file:
  csvFile = csv.reader(file)
  for lines in csvFile:
    print(lines)
    for i in lines:
      print(i)
    break

In [ ]:
def check_if_current_flow_conserved(df: pd.DataFrame) -> bool:

  # first basic check if the dataframe has 3 columns.
  assert df.shape[1] == 3, (
    f"The connectivity dataframe has {df.shape[1]}"
    f" columns, exactly 3 are required."
  )
  
  # check if all columns are named as they should
  for i in ['start','end','current_weight']:
    assert i in df.columns, (
      f"The connectivity dataframe does not contain" 
      f" expected column of name {i}."
    )

  # check all numbers in columns two and three
  combined_list = pd.concat([df.start, df.end], axis=0).unique()

  for i in combined_list:
    # check how many goes in and out
    current_in_out_balanced = (
        sum(df.loc[df.end == i,'current_weight']) - 
        sum(df.loc[df.start == i,'current_weight'])
      ) == 0
    assert current_in_out_balanced, (
        f'Current flowing in and out '
        f'of node with index {i} '
        f'is not conserved. '
    )


In [ ]:
check_if_current_flow_conserved(df)